In [1]:
from engine import PatentRetrievalService

c:\Users\SamOz.py\OneDrive\Documents\VSCode\datenna\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
service = PatentRetrievalService('../data.txt')

Batches: 100%|██████████| 149/149 [02:01<00:00,  1.23it/s]


In [6]:
keywords = ["enzymes"]

In [ ]:
# Balanced mode
results = service.retrieve_patents(keywords, precision_recall_balance=0.5)

In [8]:
results

{'metadata': {'keywords': ['enzymes'],
  'total_matches': 4703,
  'precision_recall_balance': 0.991,
  'embedding_model': 'paraphrase-multilingual-MiniLM-L12-v2'}}

In [33]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'

dataset_path = '../data.txt'
with open(dataset_path, 'r', encoding='utf-8') as f:
        abstracts = [line.strip() for line in f if line.strip()]

embedding_model = SentenceTransformer(model_name)

In [34]:
embeddings = embedding_model.encode(sentences=abstracts, show_progress_bar=True)

Batches: 100%|██████████| 149/149 [02:02<00:00,  1.22it/s]


In [35]:
faiss.normalize_L2(embeddings)

In [36]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product index for cosine similarity
index.add(embeddings)

In [106]:
keyword = ["enzymes", "medical"]
# keyword_embeddings = embedding_model.encode(keyword)
keyword_embedding = embedding_model.encode([' '.join(keywords)])

In [108]:
keyword_embedding.shape

(1, 384)

In [77]:
pooled_keyword_embeddings = np.mean(keyword_embeddings, axis=0).reshape(1, -1)

In [79]:
faiss.normalize_L2(pooled_keyword_embeddings)

In [121]:
distances, indices = index.search(pooled_keyword_embeddings, k=len(abstracts))

In [142]:
precision_recall_balance = 0.001
threshold = np.percentile(
            distances, 
            (1 - precision_recall_balance) * 100
        )
threshold

np.float32(0.48743364)

In [143]:
relevant_indices = indices[distances >= threshold]
relevant_indices

array([2734,  392, 3571, 2270, 2204])

In [135]:
distances

array([[ 0.5001985 ,  0.4993974 ,  0.49889547, ..., -0.10678346,
        -0.10801981, -0.13303286]], dtype=float32)

In [131]:
distances[0][2734]

np.float32(0.10021333)

In [124]:
for idx in relevant_indices:
    print(abstracts[idx])

Изобретение относится к области биотехнологии, конкретно, к фармацевтической композиции и способу профилактики и лечения недостаточности лизосомального фермента у субъекта, что может быть использовано в медицине. Настоящее изобретение относится к соединению HIR-Fab-IDS, применяемому для профилактики или лечения недостаточности лизосомального фермента у субъекта, имеющего лизосомную болезнь накопления, представляющую собой мукополисахаридоз II типа, где субъекту вводят по меньшей мере одну дозу соединения HIR-Fab-IDS от 1 до 12 мг/кг, к фармацевтической композиции для применения для профилактики или лечения недостаточности лизосомального фермента у субъекта, имеющей лизосомную болезнь накопления, представляющую собой мукополисахаридоз II типа, где композиция содержит соединение HIR-Fab-IDS, а также к способу профилактики или лечения недостаточности лизосомального фермента у субъекта, имеющего лизосомную болезнь накопления, представляющую собой мукополисахаридоз II типа, включающий введе

In [136]:
results = {'abstract' : [],
        'relevance_score': [],
        }
for i, idx in enumerate(relevant_indices):
    results['abstract'].append(abstracts[idx])
    results['relevance_score'].append(distances[0][i])

In [148]:
explanation = _generate_explanation(2734, keyword, 0.001)
type(explanation)

str

In [ ]:
from typing import List
def _generate_explanation(
        abstract_idx: int,
        keywords: List[str],
        precision_recall_balance: float
    ) -> str:
        """Generate an explanation for why an abstract is relevant"""
        abstract = abstracts[abstract_idx]
        
        # Find keyword matches
        keyword_matches = [
            kw for kw in keywords 
            if kw.lower() in abstract.lower()
        ]
        
        # Compute semantic similarity
        keyword_embedding = embedding_model.encode(keywords)
        abstract_embedding = embeddings[abstract_idx]
        semantic_similarities = [
            float(np.dot(keyword_emb, abstract_embedding)) 
            for keyword_emb in keyword_embedding
        ]
        
        # Add precision-recall context to explanation
        precision_context = (
            "high precision mode - focusing on closest matches"
            if precision_recall_balance > 0.7
            else "balanced precision-recall"
            if 0.3 <= precision_recall_balance <= 0.7
            else "high recall mode - including broader matches"
        )
        
        explanation = {
            "Match Analysis": precision_context,
            "Direct keyword matches": len(keyword_matches),
            "Semantic similarity score": np.mean(semantic_similarities)
            "Matched keywords": "".join(keyword_matches) if keyword_matches else None
        }
        return explanation